# Проект по А/B-тестированию

## Задачи проекта

Наша задача — провести оценку результатов A/B-теста. В нашем распоряжении есть датасет с действиями пользователей, техническое задание и несколько вспомогательных датасетов.

1. Оценим корректность проведения теста.

2. Проанализируем результаты теста.

Чтобы оценить корректность проведения теста, проверим:

- пересечение тестовой аудитории с конкурирующим тестом;

- совпадение теста и маркетинговых событий, другие проблемы временных границ теста.

**Техническое задание (далее - ТЗ)**

Название теста: recommender_system_test;

Группы: А (контрольная), B (новая платёжная воронка);

Дата запуска: 2020-12-07;

Дата остановки набора новых пользователей: 2020-12-21;

Дата остановки: 2021-01-04;

Аудитория: 15% новых пользователей из региона EU;

Назначение теста: тестирование изменений, связанных с внедрением улучшенной рекомендательной системы;
Ожидаемое количество участников теста: 6000.

Ожидаемый эффект: за 14 дней с момента регистрации в системе пользователи покажут улучшение каждой метрики не менее, чем на 10%:
конверсии в просмотр карточек товаров — событие product_page

просмотры корзины — product_cart

покупки — purchase.

Загрузите данные теста, проверьте корректность его проведения и проанализируйте полученные результаты.

## Изучение и предобработка данных

Импортируем библиотеки и читаем данные датафреймов.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats as st
import math as mth

import warnings
warnings.filterwarnings('ignore')

In [ ]:
ab_project_marketing_events = pd.read_csv('https://code.s3.yandex.net/datasets/ab_project_marketing_events.csv')
final_ab_new_users = pd.read_csv('https://code.s3.yandex.net/datasets/final_ab_new_users.csv')
final_ab_events = pd.read_csv('https://code.s3.yandex.net/datasets/final_ab_events.csv')
final_ab_participants = pd.read_csv('https://code.s3.yandex.net/datasets/final_ab_participants.csv')

In [ ]:
ab_project_marketing_events.head(5)

In [ ]:
ab_project_marketing_events.info()

In [ ]:
ab_project_marketing_events.duplicated().sum()

In [ ]:
ab_project_marketing_events['start_dt'] = pd.to_datetime(ab_project_marketing_events['start_dt'], 
                                    format='%Y.%m.%d')


ab_project_marketing_events['finish_dt'] = pd.to_datetime(ab_project_marketing_events['finish_dt'], 
                                    format='%Y.%m.%d')

In [ ]:
final_ab_new_users.head(5)

In [ ]:
final_ab_new_users.info()

In [ ]:
final_ab_new_users['first_date'] = pd.to_datetime(final_ab_new_users['first_date'], 
                                    format='%Y.%m.%d')

In [ ]:
final_ab_events.head(5)

In [ ]:
final_ab_events.info()

In [ ]:
final_ab_events['event_dt'] = pd.to_datetime(final_ab_events['event_dt'], 
                                    format='%Y.%m.%d %H:%M:%S')

In [ ]:
final_ab_participants.head(5)

In [ ]:
final_ab_participants.info()

In [ ]:
all_df = [ab_project_marketing_events, final_ab_new_users,  final_ab_events, final_ab_participants]

In [ ]:
for table in all_df:
    print(table.isnull().sum())

Много пропусков в датасете events в колонке details, но это дополнительные данные о событии, которых вполне может и не быть. Для удобства заменим их на нули

In [ ]:
final_ab_events = final_ab_events.fillna(0)

In [ ]:
final_ab_events.isnull().sum()

In [ ]:
for table in all_df:
    print(table.duplicated().sum())

**Промежуточный вывод:**

- Заменили типы данных в столбцах с датами
- Проверили дубликаты
- Проверили пропуски
- Проставили 0 в местах пропуска

## Оценка корректности проведения теста.

**Проверим группы на пересекающихся пользователей**

In [ ]:
cross_users = final_ab_participants.groupby('user_id')['group'].nunique().reset_index()
cross_users = cross_users.query('group > 1')
len(cross_users)

776 пользователей находится в обеих группах

**Проверим пересечение в группах у теста recommender_system_test**.

Воспользуемся методом intersect1d, который возвращает отсортированные уникальные значения в обоих входных массивах

In [ ]:
intersecting = final_ab_participants[final_ab_participants['ab_test'] == 'recommender_system_test']

In [ ]:
len(np.intersect1d
    (intersecting[intersecting['group'] == 'A']['user_id'], 
     intersecting[intersecting['group'] == 'B']['user_id']
    )
   )

Внутри теста в группах пересекающихся пользователей нет

**Проверим пересечение у двух имещихся тестов**

In [ ]:
final_ab_participants['ab_test'].unique()

In [ ]:
int_1 = len(np.intersect1d
    (final_ab_participants.query('ab_test == "recommender_system_test"')['user_id'].unique(), 
     final_ab_participants.query('ab_test == "interface_eu_test"')['user_id'].unique()
    )
   )
int_1

Между двумя нашими тестами имеется 1602 общих пользователя

**Проверим пользователей на даты регистрации для А/В-теста**

In [ ]:
print("Минимальная дата регистрации пользователей: ", final_ab_new_users['first_date'].min())
print("Максимальная дата регистрации пользователей: ", final_ab_new_users['first_date'].max())

Для нашего датасета соответсвующего ТЗ, не станем учитывать пользователей зарегистрировавшихся после 21 декабря

In [ ]:
actual_final_ab_new_users = final_ab_new_users.query('first_date <= "2020-12-21"')
actual_final_ab_new_users.sort_values(by='first_date', ascending=True).tail(5)

In [ ]:
actual_final_ab_new_users['user_id'].nunique()

**Проверим даты проведения А/В-теста**

In [ ]:
print("Минимальная дата действий пользователей: ", final_ab_events['event_dt'].min())
print("Максимальная дата действий пользователей: ", final_ab_events['event_dt'].max())

Дата проведения теста по ТЗ с 2020-12-07 по 2021-01-04, по факту же максимальная дата действий пользователей составляет 2020-12-30. Возможно данные не полные, либо тест свернули раньше планируемого.

**Посмотрим какие мероприятия проходили в период нашего теста**

In [ ]:
ab_project_marketing_events.query('start_dt < "2021-12-30" and finish_dt > "2020-12-07"')

В это время проходило 2 маркетинговых мероприятия, затронувшие регионы EU, N.America и CIS

**Проверим на сколько точно в ТЗ перечислены показатели аудитории**

В ТЗ сказано: "Аудитория: 15% новых пользователей из региона EU;". Формулировка вызывает вопросы. Будем считать, что тут имеется в виду процент количества тестируемых пользователей из Европы относительно общего количества пользователей из Европы, как то так

In [ ]:
recommender_ab_participants = final_ab_participants.query('ab_test == "recommender_system_test"')

In [ ]:
EU_users = actual_final_ab_new_users.query('region == "EU"')
EU_users.head(5)

In [ ]:
all_ab_users = recommender_ab_participants.merge(final_ab_new_users, how='left', on='user_id')

In [ ]:
region_users_count = (
    all_ab_users.groupby('region')
    .agg(count = ('user_id', 'count')).sort_values(by = 'count', ascending = False).reset_index()
             )
region_users_count

In [ ]:
print('Процент количества тестируемых пользователей из Европы относительно общего количества пользователей из Европы: {:.2%}'.
      format(region_users_count['count'][0] / EU_users['user_id'].nunique()))

**Соединим наши 2 таблицы с пользователями с таблицей с событиями новых пользователей, чтобы посмотреть какие события были затронуты и сколько пользователей осталось**

In [ ]:
temp_df = recommender_ab_participants.merge(actual_final_ab_new_users, on='user_id', how='left')
final_ab_df = temp_df.merge(final_ab_events, on='user_id', how='left')
final_ab_df

Проверим таблицу на пропуски и дубли и избавимся от них

In [ ]:
final_ab_df.isna().sum()

In [ ]:
final_ab_df.duplicated().sum()

In [ ]:
final_ab_df.isna().sum()

In [ ]:
final_ab_df = final_ab_df.dropna(subset=['event_dt', 'event_name'])

Посчитаем lifetime и уберём события, что старше 14 дней с момента регистрации пользователя, как написано в ТЗ.

In [ ]:
final_ab_df['lifetime'] = final_ab_df['event_dt'] - final_ab_df['first_date']
final_ab_df = final_ab_df[final_ab_df['lifetime'] <= '14 days']
final_ab_df

In [ ]:
final_ab_df['user_id'].nunique()

**Промежуточный итог:**

- Пересечения между группами, как таковыми есть есть и их 776
- Пересечений пользователей в группах recommender_system_test найти не удалось
- Пересечения пользователей между тестами так же удалось обнаружить в количестве 1 602 штук
- Даты регистрации пользователей не соотвествовали ТЗ, поэтому убрали тех, кто не попал в период
- Про 6 000 пользователей в ТЗ тоже наврали, т.к. по остальным условиям ТЗ мы имеем только 3 675 уникальных пользователей
- Создали отделный датафрейм, куда записали всех участников АБ-теста в соответсвии со сроками по ТЗ

Уже закрадываются подозрения в полезности результатов подобного теста. Но исследование, есть исследование, так что мы продолжим

## Исследовательский анализ данных

### Количество событий на пользователя одинаково распределены в выборках?

In [ ]:
group_a = (
    final_ab_df.query('group == "A"')
    .groupby(['user_id', 'event_name'])['event_name'].count().to_frame()
    .rename(columns = {'event_name':'count'})
    .sort_values(by = 'count')
    .reset_index()
)
group_a

In [ ]:
group_b = (
    final_ab_df.query('group == "B"')
    .groupby(['user_id', 'event_name'])['event_name'].count().to_frame()
    .rename(columns = {'event_name':'count'})
    .sort_values(by = 'count')
    .reset_index()
)
group_b

In [ ]:
group_a_mean = group_a.groupby('event_name')['count'].mean().reset_index().round(3)
group_b_mean = group_b.groupby('event_name')['count'].mean().reset_index().round(3)

In [ ]:
group_a_mean

In [ ]:
group_b_mean

In [ ]:
fig = go.Figure(
        data = [
            go.Bar(name = 'Group - A', 
                   x = group_a_mean.event_name.to_list(),
                   y = group_a_mean['count'].to_list(),
                   text = group_a_mean['count'].to_list(),
                   textposition = 'auto'),

            go.Bar(name = 'Group - B', 
                   x = group_b_mean.event_name.to_list(),
                   y = group_b_mean['count'].to_list(),
                   text = group_b_mean['count'].to_list(),
                   textposition = 'auto')
])

fig.update_layout(title = 'Количество событий на пользователя',
                  xaxis_title = 'Наименование события',
                  yaxis_title = 'Количество событий')
fig.show()

In [ ]:
a = final_ab_df.query('group == "A"').groupby('user_id')['event_name'].count().reset_index()
b = final_ab_df.query('group == "B"').groupby('user_id')['event_name'].count().reset_index()

In [ ]:
a.describe()

In [ ]:
b.describe()

In [ ]:
plt.figure(figsize = (15,5))
sns.distplot(a['event_name'], label = 'A', kde=False)
sns.distplot(b['event_name'], label = 'B', kde=False)
plt.title('Распределение событий на каждого пользователя относительно группы')
plt.ylabel('Количество пользователей')
plt.xlabel('Количество событий')
plt.legend()
plt.show()

In [ ]:
df_1 = final_ab_df.groupby('user_id').agg(count = ('event_name', 'count')).reset_index()
df_1 = df_1.merge(final_ab_df, how='left', on='user_id')[['user_id', 'count', 'group']]
df_1.describe()

In [ ]:
plt.figure(figsize=(15,5))
plt.title('Распределение событий на пользователя относительно групп')
sns.distplot(df_1.query('group == "A"')['count'], label='A', kde=False)
sns.distplot(df_1.query('group == "B"')['count'], label='B', kde=False)
plt.legend()
plt.ylabel('Количество пользователей')
plt.xlabel('Количество событий')
plt.show()

Теперь взглянем на распределение событий внутри групп

In [ ]:
fig = px.bar(final_ab_df.groupby(['group', 'event_name'])['user_id'].nunique().reset_index(),
             x = 'group',
             y = 'user_id',
             text = 'user_id',
             color = 'event_name'
            )

fig.update_layout(title = 'Распределение событий внутри групп А и В',
                  xaxis_title = 'Группа',
                  yaxis_title = 'Количество событий',
                  width = 700,
                  height = 600)
fig.show()

**Промежуточный итог:**

- События в группах А и В распределены не одинаково. По всему видно, что и среднее и медианное количество событий больше в группе A. И в целом, у группы А гораздо больше событий чем у пользователей из группы В

### Как число событий в выборках распределено по дням?

In [ ]:
final_ab_df['dt'] = final_ab_df['event_dt'].dt.date
event_per_day = final_ab_df.groupby(['dt', 'group'])['event_name'].count().reset_index()
event_per_day = event_per_day.rename(columns={'event_name':'events'})
event_per_day.head(6)

In [ ]:
plt.figure(figsize = (15, 5))
sns.lineplot(x = 'dt', y = 'events', data=event_per_day.query('group == "A"'), label = 'A')
sns.lineplot(x = 'dt', y = 'events', data=event_per_day.query('group == "B"'), label = 'B')
plt.title('Распределение событий в выборках по дням')
plt.xlabel('День')
plt.ylabel('Количество событий')
plt.legend()
plt.show()

**Промежуточный итог:**

- По дням данные распределены не равномерно. Особенно сильно отличается группа А имея достаточно сильные взлёты показателя 13 декабря (примерно 1 100 событий) и 21 декабря(примерно 2 000 событий), хотя до этого держались в районе 300 и 1300 событий соответсвенно.
- Группа В на фоне своих товарищей по тестам выглядят куда более стабильными. Их события на протяжении всего периода колышатся в диапазоне примерно от 100 событий до 400 событий

### Как меняется конверсия в воронке в выборках на разных этапах?

In [ ]:
final_ab_df['event_name'].value_counts()

Исходя из количества событий, воронка имеет следующий вид login -> product_page -> purchase -> product_cart. Что довольно странно, но видимо у пользователей была возможность совершать покупку не переходя в корзину, оттого так и живём

Сделаем 2 группировки по А и В группам и посмотрим конверсию уникальных пользователей на каждом этапе

In [ ]:
# создадим воронку для группы А
funnel_group_A = final_ab_df.query('group =="A"')
funnel_group_A = funnel_group_A.groupby(['event_name'])['user_id']\
                .nunique().reset_index().sort_values(by = 'user_id', ascending=False)


# создадим воронку для группы B
funnel_group_B = final_ab_df.query('group =="B"')
funnel_group_B = funnel_group_B.groupby(['event_name'])['user_id']\
                .nunique().reset_index().sort_values(by = 'user_id', ascending=False)

Визуализируем воронку событий.

In [ ]:
fig = go.Figure()
new_index = [0,2,1,3]
funnel_group_A = funnel_group_A.reindex(new_index)
new_index = [0,2,1,3]
funnel_group_B = funnel_group_B.reindex(new_index)

fig.add_trace(go.Funnel(
    name = 'group A',
    y = funnel_group_A['event_name'],
    x = funnel_group_A['user_id'],
    textinfo = "value+percent previous + percent initial"
    ))

fig.add_trace(go.Funnel(
    name = 'group B',
    y = funnel_group_B['event_name'],
    x = funnel_group_B['user_id'],
    textinfo = "value+percent previous + percent initial"
    ))
fig.update_layout(
    title = {
        'text': "Воронка событий по группам А и В"})
fig.show()

Воронка группы А

100.0% -> 64.8% -> 30% -> 31.7%

Воронка группы В

100.0% -> 56.4% -> 27.5% -> 27.6%

Согласно ТЗ за 14 дней с момента регистрации, пользователи должны были показать улучшение каждой метрики не менее чем на 10%. Но ТЗ, это ТЗ, а у реальности нету чёткого ТЗ

### Какие особенности данных нужно учесть, прежде чем приступать к A/B-тестированию?

Нужно определить конкретную цель, которую хотите достичь путём тестирования; разработать и проверить рабочие гипотезы; правильно рассмотреть размеры выборки(а не как в этот раз); определить наилучшие даты тестирования (чтобы не совпадали с маркетинговыми компаниями например); выбрать подходящие метрики; и проанализировать результаты тестирования

## Оценим результаты А/В-тестирования.

### Что можно сказать про результаты A/В-тестирования?

Допущены множественные ошибки при проведении теста, халатно не соблюдено собственное ТЗ. Доверять результатам данного А/B теста не стоит

### Проверим статистическую разницу долей z-критерием

Сформулируем нулевую и альтернативную гипотезы:
- H0 - между группами А и В нет различий в конверсии;
- Н1 - между группами А и В есть различие в конверсии;

Сделаем сводную таблицу по группам и количеству событий на каждую из двух групп пользователей

In [ ]:
ab_group_events = (
    final_ab_df.pivot_table(
    index = 'event_name',
    columns = 'group',
    values = 'user_id',
    aggfunc = 'nunique')
    .sort_values(by = 'A', ascending=False).reset_index()
)
ab_group_events

In [ ]:
group_users = final_ab_df.groupby('group')['user_id'].nunique()
group_users

Воспользуемся z-критерием по долям для написания функции, которую используем относительно событий в наших группах

In [ ]:
def z_test(group_A, group_B, alpha=0.05):
    for i in ab_group_events.index:
        p1 = ab_group_events[group_A][i] / group_users[group_A]
        p2 = ab_group_events[group_B][i] / group_users[group_B]
        
        print(ab_group_events[group_A][i], ab_group_events[group_B][i], group_users[group_A],group_users[group_B] )
        
        p_combined = (ab_group_events[group_A][i] + ab_group_events[group_B][i]) / (group_users[group_A] + group_users[group_B])
        difference = p1 - p2
        z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1 / group_users[group_A] + 1 / group_users[group_B]))
        
        distr = st.norm(0, 1)
        p_value = (1 - distr.cdf(abs(z_value))) * 2
        
        print('{} p-value: {}'.format(ab_group_events['event_name'][i], p_value))
        if (p_value < alpha):
            print('Отвергаем нулевую гипотезу, между выборками есть статистически значимые различия')
        else:
            print('Не получилось отвергнуть нулевую гипотезу, статистически значимых различий в выборках нет')

In [ ]:
z_test('A', 'B')

## Общий вывод

- События в группах А и В распределены не одинаково. У группы А гораздо больше событий чем у пользователей из группы В. Возможно это произошло из за ошибок при формировании изначальной выборки. Но и в целом, группа А больше, что так же могло повлиять на результат в эту сторону.
- Распределение событий по дням у группы В вполне стремится к нормальному. У группы А же имеются несколько выбросов, так что трудно сказать достоверно относительно нормальности распределения
- Воронка группы А демонстрирует лучший показатель на последнем этапе по сравнению с группой В. Более того, ожидаемый результат в виде увеличения конверсии на не менее чем 10% по каждой метрике не оправдался
- Корректность проведения данного теста вызывает сильные сомнения. Методология проведения А/В тестирования была нарушена, собственное ТЗ попрано. Как следствие, полученные для анализа данные оказались искажены, т.е. все результаты рассчётов не могут быть использованы для принятия решений